In [2]:
import streamlit as st
from langchain.chains import LLMChain, SimpleSequentialChain
import pandas as pd
import sqlite3
import os
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from subs.db_connections import connetc_to_irish_db

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Check current working directory.
print("Current Working Directory: ", os.getcwd())


# #llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
db = connetc_to_irish_db()



Current Working Directory:  /Users/saeed/Documents/GitHub/ireland_res_chatbot


In [4]:
from subs.prompts import *



# Example formatted prompt
prompt_val = invoke_full_prompt(chain_id="chain_1")


In [5]:
prompt_val

ChatPromptTemplate(input_variables=['agent_scratchpad', 'dialect', 'input', 'top_k'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=FewShotPromptTemplate(input_variables=['dialect', 'top_k'], example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10f6b2b60>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], template='User input: {input}\nSQL query: {query}'), suffix='', prefix='You are an agent designed to interact with a SQL database.\n        Given an input question, create a syntactically correct {dialect} query to 

In [3]:
from subs.agent import sql_agent

agent = sql_agent( prompt=prompt_val)


In [4]:
agent.invoke({"input": "what is average of wind generation in each month in 2022?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT AVG(`IE Wind Generation` + `NI Wind Generation`) AS AverageWindGeneration, strftime('%Y-%m', `DateTime`) AS MonthYear FROM energy_data WHERE `DateTime` LIKE '2022%' GROUP BY MonthYear;"}`


[(1681.7478245967789, '2022-01'), (2776.5118084077367, '2022-02'), (1564.5947624495348, '2022-03'), (1426.8548583333356, '2022-04'), (1428.130300739246, '2022-05'), (1244.942120138891, '2022-06'), (856.9325806451609, '2022-07'), (822.9017184139785, '2022-08'), (1081.6955940972225, '2022-09'), (2047.5043134228179, '2022-10'), (2241.7916909722217, '2022-11'), (1646.4242170698897, '2022-12')]The average wind generation in each month in 2022 is as follows:
- January 2022: 1681.75
- February 2022: 2776.51
- March 2022: 1564.59
- April 2022: 1426.85
- May 2022: 1428.13
- June 2022: 1244.94
- July 2022: 856.93
- August 2022: 822.90
- September 2022: 1081.70
- October 2022: 2047.50
- November 2022: 2241.79
- Decem

{'input': 'what is average of wind generation in each month in 2022?',
 'output': 'The average wind generation in each month in 2022 is as follows:\n- January 2022: 1681.75\n- February 2022: 2776.51\n- March 2022: 1564.59\n- April 2022: 1426.85\n- May 2022: 1428.13\n- June 2022: 1244.94\n- July 2022: 856.93\n- August 2022: 822.90\n- September 2022: 1081.70\n- October 2022: 2047.50\n- November 2022: 2241.79\n- December 2022: 1646.42'}

In [2]:
# prompt_val.invoke()

prompt_val_2 = prompt_val.invoke(
    {
        "input": "How many years are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val_2.to_string())

System: You are an agent designed to interact with a SQL database.
        Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
        Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
        You can order the results by a relevant column to return the most interesting examples in the database.
        Query for the relevant columns needed to answer the question comprehensively. 
        In cases where calculations or comparisons across multiple sectors (such as IE and NI) are required,
        ensure to include all necessary columns for these calculations. Prioritize efficiency and relevance in your queries.
        You have access to tools for interacting with the database.
        Only use the given tools. Only use the information returned by the tools to construct your final answer.

        You MUST double check your query 

In [8]:
response

{'input': 'daily data for 2023?',
 'output': 'I cannot share with you the entire dataset for 2023; however, I can provide you with a sample of daily data for 2023:\n\n1. Date: 2023-01-01, Various energy generation and flow values\n2. Date: 2023-01-02, Various energy generation and flow values\n3. Date: 2023-01-03, Various energy generation and flow values\n4. Date: 2023-01-04, Various energy generation and flow values\n5. Date: 2023-01-05, Various energy generation and flow values\n6. Date: 2023-01-06, Various energy generation and flow values\n7. Date: 2023-01-07, Various energy generation and flow values\n8. Date: 2023-01-08, Various energy generation and flow values\n9. Date: 2023-01-09, Various energy generation and flow values\n10. Date: 2023-01-10, Various energy generation and flow values\n\nThese are just samples of the daily data for 2023.'}

In [1]:
from subs.agent import configure_sequential_chain
chain_Final = configure_sequential_chain(chain_id_sql = "chain_1",chain_id_response_plot = "chain_2")

response_of_chain = chain_Final.run("what is SNSP from January to March 2022 per month?")

/Users/saeed/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...


> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': "SELECT SUBSTR(`DateTime`, 1, 7) AS Month, AVG(`SNSP`) AS Average_SNSP FROM energy_data WHERE `DateTime` BETWEEN '2022-01-01' AND '2022-03-31' GROUP BY Month;"}`


[('2022-01', 0.350023051075269), ('2022-02', 0.5431183035714271), ('2022-03', 0.3535211057023637)]The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:
- January 2022: 0.350
- February 2022: 0.543
- March 2022: 0.354

> Finished chain.
The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:
- January 2022: 0.350
- February 2022: 0.543
- March 2022: 0.354
{"plot":{"line": {"columns": ["January", "February", "March"], "data": [0.350, 0.543, 0.354]}}
, "prompt": {"output_of_chain1": "The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:\n- January 2022: 0.350\n- Febru

In [2]:
response_of_chain

'{"plot":{"line": {"columns": ["January", "February", "March"], "data": [0.350, 0.543, 0.354]}}\n, "prompt": {"output_of_chain1": "The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:\\n- January 2022: 0.350\\n- February 2022: 0.543\\n- March 2022: 0.354"}}'

In [4]:
import json
json.loads(response_of_chain)

{'plot': {'line': {'columns': ['January', 'February', 'March'],
   'data': [0.35, 0.543, 0.354]}},
 'prompt': {'output_of_chain1': 'The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:\n- January 2022: 0.350\n- February 2022: 0.543\n- March 2022: 0.354'}}

In [6]:
def post_process_chain_response(response_of_chain):

    # Convert the response string into a dictionary
    response_dict = json.loads(response_of_chain)

    # # Access the 'plot' part of the response
    plot_info = response_dict["plot"]

    # # Access the 'prompt' part of the response
    prompt_info = response_dict["prompt"]

    return plot_info, prompt_info

In [7]:
# from subs.post_processing import post_process_chain_response

plot_info, prompt_info = post_process_chain_response(response_of_chain)

In [8]:
plot_info

{'line': {'columns': ['January', 'February', 'March'],
  'data': [0.35, 0.543, 0.354]}}

In [10]:
prompt_info['output_of_chain1']

'The System Non-Synchronous Penetration (SNSP) from January to March 2022 per month is as follows:\n- January 2022: 0.350\n- February 2022: 0.543\n- March 2022: 0.354'